![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Context Types Deep Dive

## What You'll Learn

In this notebook, you'll master the four core context types that power intelligent AI agents:

1. **System Context** - The AI's role and domain knowledge
2. **User Context** - Personal profiles and preferences
3. **Conversation Context** - Dialogue history and flow
4. **Retrieved Context** - Dynamic information from external sources

You'll learn both the **theory** (what each type is and when to use it) and the **practice** (how to build and combine them effectively).

**Time to complete:** 20-25 minutes

---

## Setup

Let's start with the essentials. You'll need an OpenAI API key to run the examples.

In [2]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

---

## Understanding the Context Window Constraint

Before we dive into context types, you need to understand the fundamental limitation that shapes all context engineering decisions.

### The Token Limit Reality

Every AI model has a **context window** - a maximum amount of text it can process in a single request.

| Model | Context Window | Approximate Words |
|-------|----------------|-------------------|
| GPT-4o | 128,000 tokens | ~96,000 words |
| GPT-4o-mini | 128,000 tokens | ~96,000 words |
| Claude 3.5 Sonnet | 200,000 tokens | ~150,000 words |

**Note:** 1 token ≈ 0.75 words in English

### What Competes for This Space?

Every element of your request must fit within this limit:

```
┌─────────────────────────────────────────┐
│     CONTEXT WINDOW (128K tokens)        │
├─────────────────────────────────────────┤
│ System Instructions       │ 2,000       │
│ Tool Definitions          │ 3,000       │
│ User Profile              │ 1,000       │
│ Conversation History      │ 4,000       │
│ Retrieved Context         │ 5,000       │
│ User Query                │ 500         │
│ Response Space            │ 4,000       │
├─────────────────────────────────────────┤
│ TOTAL USED               │ 19,500       │
│ REMAINING                │ 108,500      │
└─────────────────────────────────────────┘
```

### The Core Trade-off

**Every token spent on one thing is a token NOT available for another.**

This means context engineering requires constant decision-making:
- Is this information relevant to the current query?
- Does including this improve response quality?
- Is the improvement worth the token cost?

**All three must be "yes" or don't include it.**

---

## The Four Core Context Types

Every context-aware AI system uses these four components. Let's explore each one, understand when to use it, and learn how to implement it.

Throughout this notebook, we'll build a **Redis University course advisor** that helps students choose the right courses based on their background, goals, and learning path.

---

## 1. System Context: The AI's Identity

### What Is System Context?

System context defines **what the AI is** and **what it knows**. This is static information that:
- Applies to ALL users equally
- Rarely changes (typically only with code deployments)
- Is hardcoded in your application
- Must always be present

### What Goes in System Context?

1. **Role Definition** - What is the AI's purpose?
2. **Domain Knowledge** - What information does it have?
3. **Behavioral Instructions** - How should it respond?
4. **Business Rules** - What constraints apply?

### When to Use System Context

Use system context for information that:
- ✅ Defines the agent's core identity
- ✅ Contains universal business logic
- ✅ Provides essential domain knowledge
- ✅ Should be consistent across all interactions

### Building System Context Step-by-Step

In [3]:
# Step 1: Define the AI's role
system_context = """You are a Redis University course advisor."""

This is the foundation - but it's not enough. The AI needs domain knowledge to be useful.

In [4]:
# Step 2: Add domain knowledge (available courses)
system_context = """You are a Redis University course advisor.

Available Courses:
- RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)
  Learn Redis fundamentals: strings, hashes, lists, sets, sorted sets

- RU201: Redis for Python Developers (Intermediate, 6-8 hours)
  Prerequisites: RU101, Python experience
  Build Redis applications with Python and redis-py

- RU202: Redis for Java Developers (Intermediate, 6-8 hours)
  Prerequisites: RU101, Java experience
  Build Redis applications with Java and Jedis

- RU301: Vector Similarity Search with Redis (Advanced, 8-10 hours)
  Prerequisites: RU201 or RU202, ML/AI interest
  Implement semantic search and RAG systems
"""

Now we have both role and knowledge. Finally, let's add behavioral guidance.

In [5]:
# Step 3: Add behavioral instructions and business rules
system_context = """You are a Redis University course advisor.

Available Courses:
- RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)
  Learn Redis fundamentals: strings, hashes, lists, sets, sorted sets

- RU201: Redis for Python Developers (Intermediate, 6-8 hours)
  Prerequisites: RU101, Python experience
  Build Redis applications with Python and redis-py

- RU202: Redis for Java Developers (Intermediate, 6-8 hours)
  Prerequisites: RU101, Java experience
  Build Redis applications with Java and Jedis

- RU301: Vector Similarity Search with Redis (Advanced, 8-10 hours)
  Prerequisites: RU201 or RU202, ML/AI interest
  Implement semantic search and RAG systems

Guidelines:
1. Always provide specific course recommendations with clear reasoning
2. Consider the student's background, completed courses, and interests
3. Ensure prerequisites are met before recommending advanced courses
4. Be encouraging and supportive in your guidance
"""

### Key Insight: System Context is Universal

Notice that system context doesn't mention any specific user. It's the same for everyone. Whether the student is Sarah, Alex, or anyone else, this context remains constant.

This is what makes it "static" - you write it once in your code and it's always present with a fixed token cost.

---

## 2. User Context: Personal Information

### What Is User Context?

User context contains **information about the specific user** that enables personalization. Unlike system context, this is dynamic and different for each user.

### What Goes in User Context?

1. **Profile Information** - Name, background, experience level
2. **Learning History** - Completed courses, achievements
3. **Preferences** - Learning style, time availability, interests
4. **Goals** - What the user wants to achieve

### When to Use User Context

Use user context when:
- ✅ Information is specific to an individual user
- ✅ Personalization will significantly improve responses
- ✅ The information persists across multiple sessions
- ✅ You have a reliable way to store and retrieve user data

### Building User Context Step-by-Step

In [6]:
# Step 1: Create a user profile as a dictionary
# In production, this would come from a database
sarah_profile = {
    "name": "Sarah Chen",
    "background": "Python developer, 2 years experience",
    "completed_courses": ["RU101"],
    "interests": ["machine learning", "data science", "vector search"],
    "time_availability": "evenings and weekends",
    "goal": "Build a RAG system for my company's documentation",
}

The dictionary format is great for storage, but we need to format it for the LLM.

In [7]:
# Step 2: Format as context for the LLM


def format_user_context(profile):
    """Convert user profile dictionary to formatted context string"""
    return f"""Student Profile:
- Name: {profile['name']}
- Background: {profile['background']}
- Completed Courses: {', '.join(profile['completed_courses'])}
- Interests: {', '.join(profile['interests'])}
- Availability: {profile['time_availability']}
- Goal: {profile['goal']}
"""


user_context = format_user_context(sarah_profile)
print(user_context)

Student Profile:
- Name: Sarah Chen
- Background: Python developer, 2 years experience
- Completed Courses: RU101
- Interests: machine learning, data science, vector search
- Availability: evenings and weekends
- Goal: Build a RAG system for my company's documentation



### Understanding User Context Differences

Let's create another user to see how context changes:

In [8]:
# Create a different user with different needs
alex_profile = {
    "name": "Alex Kumar",
    "background": "Java backend engineer, 5 years experience",
    "completed_courses": ["RU101", "RU202"],
    "interests": ["distributed systems", "caching", "performance optimization"],
    "time_availability": "flexible schedule",
    "goal": "Optimize database query performance with Redis caching",
}

alex_context = format_user_context(alex_profile)
print(alex_context)

Student Profile:
- Name: Alex Kumar
- Background: Java backend engineer, 5 years experience
- Completed Courses: RU101, RU202
- Interests: distributed systems, caching, performance optimization
- Availability: flexible schedule
- Goal: Optimize database query performance with Redis caching



### Key Insight: Different Users = Different Context

Notice how Sarah and Alex have:
- Different programming backgrounds (Python vs Java)
- Different completed courses
- Different interests and goals

This personalized context allows the AI to give tailored recommendations. Sarah might be guided toward RU201 and RU301, while Alex might focus on advanced caching strategies.

---

## 3. Conversation Context: Maintaining Dialogue Flow

### What Is Conversation Context?

Conversation context is the **history of the current dialogue**. It allows the AI to:
- Remember what was just discussed
- Understand references like "it" or "that course"
- Build on previous responses
- Maintain coherent multi-turn conversations

### What Goes in Conversation Context?

1. **Previous User Messages** - What the user has asked
2. **Previous AI Responses** - What the AI has said
3. **Context from Earlier in the Session** - Background established during this interaction

### When to Use Conversation Context

Always include conversation context for:
- ✅ Multi-turn conversations (more than a single Q&A)
- ✅ When users reference "it", "that", or previous topics
- ✅ When building on previous responses
- ✅ When maintaining coherent dialogue

### Building Conversation Context Step-by-Step

In [9]:
# Step 1: Start with an empty conversation history
conversation_history = []

As the conversation progresses, we add each exchange to the history.

In [10]:
# Step 2: Add the first user message
conversation_history.append(
    {"role": "user", "content": "What Redis course should I take next?"}
)

In [11]:
# Step 3: Add the AI's response (simulated)
conversation_history.append(
    {
        "role": "assistant",
        "content": """Based on your Python background and completion of RU101, 
I recommend RU201: Redis for Python Developers. This course will teach you 
how to build Redis applications using redis-py, which aligns perfectly with 
your goal of building a RAG system.""",
    }
)

In [12]:
# Step 4: Add a follow-up question that references previous context
conversation_history.append(
    {"role": "user", "content": "How long will that take me to complete?"}
)

Notice the user said "that" instead of "RU201". The AI needs the conversation history to understand what "that" refers to.

In [13]:
# Let's view the complete conversation history
for i, msg in enumerate(conversation_history, 1):
    print(f"Turn {i} ({msg['role']}):")
    print(f"{msg['content']}\n")

Turn 1 (user):
What Redis course should I take next?

Turn 2 (assistant):
Based on your Python background and completion of RU101, 
I recommend RU201: Redis for Python Developers. This course will teach you 
how to build Redis applications using redis-py, which aligns perfectly with 
your goal of building a RAG system.

Turn 3 (user):
How long will that take me to complete?



### Key Insight: Conversation History Enables Natural Dialogue

Without conversation history:
- ❌ "How long will **that** take?" → AI doesn't know what "that" refers to

With conversation history:
- ✅ "How long will **that** take?" → AI knows "that" = RU201

### Managing Context Window with Long Conversations

As conversations grow, they consume more tokens. Common strategies:

1. **Keep recent history** - Only include last N turns
2. **Summarize older context** - Compress early conversation into a summary
3. **Extract key facts** - Pull out important decisions/facts, discard the rest

---

## 4. Retrieved Context: Dynamic Information

### What Is Retrieved Context?

Retrieved context is **relevant information fetched on-demand** based on the current query. This is the most dynamic type of context - it changes with every query.

### What Goes in Retrieved Context?

1. **Database Records** - Course details, user records, etc.
2. **Search Results** - Relevant documents from vector/semantic search
3. **API Responses** - Real-time data from external services
4. **Computed Information** - Analysis or calculations performed on-demand

### When to Use Retrieved Context

Use retrieved context when:
- ✅ Information is too large to include statically
- ✅ Only a small subset is relevant to each query
- ✅ Information changes frequently
- ✅ You can retrieve it efficiently based on the query

### Building Retrieved Context Step-by-Step

In [14]:
# Step 1: Simulate a course database
# In production, this would be Redis, etc.
course_database = {
    "RU101": {
        "title": "Introduction to Redis Data Structures",
        "level": "Beginner",
        "description": "Master Redis fundamentals: strings, hashes, lists, sets, and sorted sets",
        "duration": "4-6 hours",
        "prerequisites": [],
        "topics": ["Data structures", "Basic commands", "Use cases"],
    },
    "RU201": {
        "title": "Redis for Python Developers",
        "level": "Intermediate",
        "description": "Build production Redis applications with Python and redis-py",
        "duration": "6-8 hours",
        "prerequisites": ["RU101", "Python experience"],
        "topics": ["redis-py library", "Connection pooling", "Pipelining", "Pub/Sub"],
    },
    "RU202": {
        "title": "Redis for Java Developers",
        "level": "Intermediate",
        "description": "Build production Redis applications with Java and Jedis",
        "duration": "6-8 hours",
        "prerequisites": ["RU101", "Java experience"],
        "topics": [
            "Jedis library",
            "Connection pooling",
            "Transactions",
            "Redis Streams",
        ],
    },
    "RU301": {
        "title": "Vector Similarity Search with Redis",
        "level": "Advanced",
        "description": "Implement semantic search and RAG systems with Redis vector capabilities",
        "duration": "8-10 hours",
        "prerequisites": ["RU201 or RU202", "ML/AI interest"],
        "topics": [
            "Vector embeddings",
            "Semantic search",
            "RAG architecture",
            "Hybrid search",
        ],
    },
}

Now let's simulate retrieving course information based on a query.

In [15]:
# Step 2: Create a retrieval function


def retrieve_course_info(course_code):
    """Retrieve detailed information about a specific course"""
    course = course_database.get(course_code)
    if not course:
        return None

    return f"""Course Details:
Code: {course_code}
Title: {course['title']}
Level: {course['level']}
Description: {course['description']}
Duration: {course['duration']}
Prerequisites: {', '.join(course['prerequisites']) if course['prerequisites'] else 'None'}
Topics Covered: {', '.join(course['topics'])}
"""


# Retrieve information about RU201
retrieved_context = retrieve_course_info("RU201")
print(retrieved_context)

Course Details:
Code: RU201
Title: Redis for Python Developers
Level: Intermediate
Description: Build production Redis applications with Python and redis-py
Duration: 6-8 hours
Prerequisites: RU101, Python experience
Topics Covered: redis-py library, Connection pooling, Pipelining, Pub/Sub



### Key Insight: Retrieved Context is Query-Specific

Notice that we only retrieved information about RU201 - the course the user asked about. We didn't include:
- RU101 details (user already completed it)
- RU202 details (not relevant to a Python developer)
- RU301 details (not the current focus)

This selective retrieval is what makes this approach scalable. Imagine having 500 courses - you can't include them all in every request, but you can retrieve the 2-3 most relevant ones.

---

## Bringing It All Together: Complete Context Integration

Now that we understand each context type individually, let's see how they work together to create an intelligent, personalized response.

### The Complete Picture

Here's how all four context types combine in a single LLM call:

```
┌─────────────────────────────────────────────┐
│           COMPLETE LLM REQUEST              │
├─────────────────────────────────────────────┤
│ 1. SYSTEM CONTEXT (Static)                  │
│    - Role: "You are a course advisor"       │
│    - Domain: Available courses              │
│    - Rules: Guidelines and constraints      │
├─────────────────────────────────────────────┤
│ 2. USER CONTEXT (Dynamic - User Specific)   │
│    - Profile: Sarah Chen, Python dev        │
│    - History: Completed RU101               │
│    - Goal: Build RAG system                 │
├─────────────────────────────────────────────┤
│ 3. CONVERSATION CONTEXT (Dynamic - Session) │
│    - User: "What course should I take?"     │
│    - AI: "I recommend RU201..."             │
│    - User: "How long will that take?"       │
├─────────────────────────────────────────────┤
│ 4. RETRIEVED CONTEXT (Dynamic - Query)      │
│    - RU201 course details                   │
│    - Duration, prerequisites, topics        │
├─────────────────────────────────────────────┤
│ RESULT: Personalized, context-aware answer  │
└─────────────────────────────────────────────┘
```

### Let's Build This Step-by-Step

In [16]:
# Step 1: Start with system context
messages = [{"role": "system", "content": system_context}]

In [17]:
# Step 2: Add user context
messages.append({"role": "system", "content": user_context})

In [18]:
# Step 3: Add conversation history
messages.extend(conversation_history)

In [19]:
# Step 4: Add retrieved context
messages.append({"role": "system", "content": retrieved_context})

In [20]:
messages

[{'role': 'system',
  'content': "You are a Redis University course advisor.\n\nAvailable Courses:\n- RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)\n  Learn Redis fundamentals: strings, hashes, lists, sets, sorted sets\n\n- RU201: Redis for Python Developers (Intermediate, 6-8 hours)\n  Prerequisites: RU101, Python experience\n  Build Redis applications with Python and redis-py\n\n- RU202: Redis for Java Developers (Intermediate, 6-8 hours)\n  Prerequisites: RU101, Java experience\n  Build Redis applications with Java and Jedis\n\n- RU301: Vector Similarity Search with Redis (Advanced, 8-10 hours)\n  Prerequisites: RU201 or RU202, ML/AI interest\n  Implement semantic search and RAG systems\n\nGuidelines:\n1. Always provide specific course recommendations with clear reasoning\n2. Consider the student's background, completed courses, and interests\n3. Ensure prerequisites are met before recommending advanced courses\n4. Be encouraging and supportive in your guidance\

### Making the Complete LLM Call

In [21]:
# Make the LLM call with complete context
response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, temperature=0.7
)

answer = response.choices[0].message.content
print("AI Response:")
print(answer)

AI Response:
RU201: Redis for Python Developers will take you approximately 6 to 8 hours to complete. Since you can dedicate time during evenings and weekends, you can spread the course over a few sessions to make it manageable and absorb the material effectively. Enjoy your learning experience!


### What Just Happened?

The LLM received all four context types and used them to generate a personalized response:

1. **System Context** told it to act as a course advisor and provided course information
2. **User Context** gave it Sarah's background, interests, and goals
3. **Conversation Context** showed that "that" refers to RU201
4. **Retrieved Context** provided detailed information about RU201's duration and topics

The result is a response that:
- Understands what course "that" refers to
- Considers Sarah's available time (evenings and weekends)
- Relates the duration to her specific situation
- Stays aligned with her goal of building a RAG system

---

## Context Management Strategies

Different situations require different approaches to context management. Let's explore three common strategies.

### Strategy 1: New User (Minimal Context)

**Scenario:** First-time user, no conversation history

| Context Type | What to Include | Token Budget |
|--------------|-----------------|-------------|
| System | Full role definition and course catalog | 2,000 |
| User | Basic profile only (if available) | 500 |
| Conversation | Empty (new session) | 0 |
| Retrieved | General overview information | 1,000 |
| **Total** | | **3,500** |

**Use when:**
- First interaction with a user
- No user history available
- Providing general guidance

### Strategy 2: Returning User (Rich Context)

**Scenario:** User with history, ongoing conversation

| Context Type | What to Include | Token Budget |
|--------------|-----------------|-------------|
| System | Full role definition and course catalog | 2,000 |
| User | Complete profile + learning history | 1,500 |
| Conversation | Last 5-10 turns of dialogue | 3,000 |
| Retrieved | Personalized, highly relevant course details | 2,000 |
| **Total** | | **8,500** |

**Use when:**
- User has established history
- Multi-turn conversation in progress
- Deep personalization is valuable

### Strategy 3: Long Conversation (Optimized Context)

**Scenario:** Approaching token limits, need to optimize

| Context Type | What to Include | Token Budget |
|--------------|-----------------|-------------|
| System | Condensed role + essential rules only | 1,000 |
| User | Key profile facts only | 500 |
| Conversation | Summarized key decisions + last 3 turns | 2,000 |
| Retrieved | Only the most relevant details | 1,000 |
| **Total** | | **4,500** |

**Use when:**
- Conversation has many turns
- Approaching context window limit
- Need to maintain performance

### Implementing an Adaptive Strategy

In [22]:
def build_context_adaptively(user_profile, conversation_history, query):
    """
    Build context adaptively based on conversation length
    """
    # Count conversation tokens (rough estimate)
    conv_tokens = sum(len(msg["content"].split()) * 1.3 for msg in conversation_history)

    messages = []

    # Strategy selection based on conversation length
    if len(conversation_history) == 0:
        # New user - full system context
        messages.append({"role": "system", "content": system_context})
        if user_profile:
            messages.append(
                {"role": "system", "content": format_user_context(user_profile)}
            )

    elif conv_tokens < 10000:
        # Normal conversation - rich context
        messages.append({"role": "system", "content": system_context})
        messages.append(
            {"role": "system", "content": format_user_context(user_profile)}
        )
        messages.extend(conversation_history)

    else:
        # Long conversation - optimized context
        # Use condensed system context
        condensed_system = "You are a Redis University course advisor. Help students choose appropriate courses."
        messages.append({"role": "system", "content": condensed_system})

        # Include only key user facts
        key_facts = f"Student: {user_profile['name']}, {user_profile['background']}. Completed: {', '.join(user_profile['completed_courses'])}"
        messages.append({"role": "system", "content": key_facts})

        # Include only recent conversation history
        messages.extend(conversation_history[-6:])

    # Always add retrieved context if relevant
    # (In production, you'd determine relevance and retrieve accordingly)

    return messages

---

## Best Practices for Context Engineering

### 1. Start Simple, Add Complexity Gradually

Don't try to build everything at once. Follow this progression:

```python
# Phase 1: Basic agent with system context only
agent = BasicAgent(system_context)

# Phase 2: Add user context
agent.set_user_profile(user_profile)

# Phase 3: Add conversation history
agent.enable_conversation_memory()

# Phase 4: Add retrieval
agent.add_retrieval_system(course_database)
```

### 2. Measure Token Usage Continuously

Always know your token consumption:

In [23]:
def estimate_tokens(text):
    """Rough token estimation (for planning purposes)"""
    return len(text.split()) * 1.3


def analyze_context_usage(messages):
    """Analyze token usage across context types"""
    total_tokens = 0
    breakdown = {}

    for msg in messages:
        tokens = estimate_tokens(msg["content"])
        total_tokens += tokens

        # Categorize by role
        role = msg["role"]
        breakdown[role] = breakdown.get(role, 0) + tokens

    print(f"Total tokens: {total_tokens:.0f}")
    print(f"Percentage of 128K limit: {total_tokens/128000*100:.1f}%")
    print("\nBreakdown:")
    for role, tokens in breakdown.items():
        print(f"  {role}: {tokens:.0f} tokens ({tokens/total_tokens*100:.1f}%)")


# Analyze our context
analyze_context_usage(messages)

Total tokens: 332
Percentage of 128K limit: 0.3%

Breakdown:
  system: 261 tokens (78.8%)
  user: 20 tokens (5.9%)
  assistant: 51 tokens (15.3%)


### 3. Optimize for Relevance, Not Completeness

**Wrong approach:** Include everything you have
```python
# Bad: Including all 50 courses = 30,000 tokens
context = "\n".join([format_course(c) for c in all_courses])
```

**Right approach:** Include only what's relevant
```python
# Good: Including only relevant courses = 2,000 tokens
relevant_courses = search_courses(query, user_profile, limit=3)
context = "\n".join([format_course(c) for c in relevant_courses])
```

### 4. Use Clear, Structured Formatting

LLMs perform better with well-structured context:

```python
# Good structure
context = """
ROLE: Course advisor for Redis University

STUDENT PROFILE:
- Name: Sarah Chen
- Background: Python developer
- Completed: RU101

RELEVANT COURSES:
- RU201: Redis for Python (6-8 hours)
  Prerequisites: RU101, Python experience

TASK: Recommend the best next course for this student.
"""
```

### 5. Test Different Context Combinations

Context engineering is empirical - always test:

In [24]:
def test_context_strategies(user_profile, test_queries):
    """
    Test different context strategies to find the best approach
    """
    strategies = [
        ("minimal", [{"role": "system", "content": system_context}]),
        (
            "with_user",
            [
                {"role": "system", "content": system_context},
                {"role": "system", "content": format_user_context(user_profile)},
            ],
        ),
        (
            "with_retrieval",
            [
                {"role": "system", "content": system_context},
                {"role": "system", "content": format_user_context(user_profile)},
                {"role": "system", "content": retrieved_context},
            ],
        ),
    ]

    for query in test_queries:
        print(f"\nQuery: {query}")
        print("=" * 60)

        for strategy_name, context_messages in strategies:
            messages = context_messages + [{"role": "user", "content": query}]

            response = client.chat.completions.create(
                model="gpt-4o-mini", messages=messages, max_tokens=150
            )

            print(f"\n{strategy_name} strategy:")
            print(response.choices[0].message.content)


# Example usage (uncomment to run)
test_queries = [
    "What course should I take next?",
    "I want to learn about vector search",
    "How long will it take to become Redis-proficient?",
]
test_context_strategies(sarah_profile, test_queries)


Query: What course should I take next?

minimal strategy:
To provide you with the best recommendation, I would need to know a bit more about your current background. Specifically:

1. Have you completed any of the available courses?
2. What level of programming experience do you have (Python, Java, etc.)?
3. Are you interested in machine learning or artificial intelligence?
4. What are your goals or what do you hope to achieve by taking the next course?

Once I have this information, I can suggest the most suitable course for you!

with_user strategy:
Hi Sarah!

Given your background as a Python developer and the fact that you've already completed RU101, you're well-prepared to dive into the next level of Redis courses. Since you have an interest in machine learning and data science, as well as a goal to build a RAG (Retrieval-Augmented Generation) system for your company's documentation, I highly recommend you take **RU201: Redis for Python Developers**.

This course will build on yo

**Example expected output:**
```
Query: What course should I take next?
============================================================

minimal strategy:
To provide you with the best recommendation, I would need to know a bit more about your current background. Specifically:

1. Have you completed any of the available courses?
2. What level of programming experience do you have (Python, Java, etc.)?
3. Are you interested in machine learning or artificial intelligence?
4. What are your goals or what do you hope to achieve by taking the next course?

Once I have this information, I can suggest the most suitable course for you!

with_user strategy:
Hi Sarah!

Given your background as a Python developer and the fact that you've already completed RU101, you're well-prepared to dive into the next level of Redis courses. Since you have an interest in machine learning and data science, as well as a goal to build a RAG (Retrieval-Augmented Generation) system for your company's documentation, I highly recommend you take **RU201: Redis for Python Developers**.

This course will build on your existing knowledge from RU101 and will teach you how to effectively use Redis to create applications specifically with Python. This is perfect for your goals, as it will give you the necessary skills to leverage Redis in developing your RAG system.

Once you complete RU201, you can then progress to **RU301

with_retrieval strategy:
Based on your background as a Python developer with two years of experience, along with your completion of RU101, I highly recommend that you take **RU201: Redis for Python Developers**. 

This course is tailored for individuals with a grounding in Python who want to leverage Redis to build applications. Since you're interested in machine learning and data science, mastering Redis with Python will significantly enhance your ability to develop applications like a RAG system for your company's documentation.

Taking RU201 will equip you with key concepts and the redis-py library, which are essential for efficiently working with Redis in your projects. With your evening and weekend availability, you should be able to complete the course within the estimated 6-8 hours.

Once you've completed RU201, you'll

Query: I want to learn about vector search
============================================================

minimal strategy:
That's great! Vector search is an exciting and increasingly important topic, especially in the fields of machine learning and artificial intelligence. To get started with vector similarity search using Redis, you'll want to take RU301: Vector Similarity Search with Redis.

Here are the prerequisites and reasoning for this recommendation:

1. **Prerequisites**: You need to have completed either RU201 (Redis for Python Developers) or RU202 (Redis for Java Developers) before taking RU301. Both of these intermediate courses cover building Redis applications and will give you a strong foundation.

2. **Interest in ML/AI**: Since you're interested in vector search, it's essential to have a background or understanding of machine learning or AI concepts, which RU301 will help you with by

with_user strategy:
Hi Sarah! It's great to see your interest in vector search, especially since you're looking to build a RAG (Retrieve and Generate) system for your company's documentation.

Given your background as a Python developer and that you've completed RU101, I recommend you take **RU201: Redis for Python Developers** next. This course will help you build Redis applications specifically with Python and teach you how to leverage Redis for your data storage needs. It's an important stepping stone before diving into advanced topics.

Once you've completed RU201, you can then move on to **RU301: Vector Similarity Search with Redis**. This advanced course will delve into implementing semantic search and other techniques that are essential for your RAG system project.

These courses align perfectly with your interests

with_retrieval strategy:
Hi Sarah! It's fantastic to see your interest in learning about vector search, especially since you're aiming to build a RAG (Retrieval-Augmented Generation) system for your company's documentation. Given your background as a Python developer and your completion of RU101, the next step for you would be to enroll in **RU201: Redis for Python Developers**.

### Here’s why RU201 is an excellent fit for you:

1. **Prerequisites Met**: You’ve already completed RU101, and as a Python developer, you have the requisite experience to succeed in this course.
2. **Focused on Python**: This course specifically teaches you how to build Redis applications with Python, which aligns perfectly with your background.
3. **Prepare for Advanced

Query: How long will it take to become Redis-proficient?
============================================================

minimal strategy:
Becoming proficient in Redis can vary greatly depending on your current background, experience, and how much time you can dedicate to learning. Here's a general guideline based on the courses available:

1. **RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)** - This foundational course will introduce you to basic Redis concepts and data structures. Completing this course is essential for starting your Redis journey.

2. **RU201: Redis for Python Developers (Intermediate, 6-8 hours)** - If you have experience with Python, this course will build on your knowledge from RU101 and teach you how to integrate Redis into Python applications. This is a great next step if you’re looking to apply Redis practically.

3. **RU

with_user strategy:
The time it takes to become proficient in Redis can vary depending on your prior knowledge, the complexity of the projects you want to undertake, and the time you can dedicate to learning. Given your background as a Python developer with two years of experience, you've already completed RU101, which gives you a solid foundation in Redis fundamentals.

Here’s a suggested pathway to proficiency based on your profile:

1. **RU101: Introduction to Redis Data Structures** - You’ve completed this course, which typically takes 4-6 hours.

2. **RU201: Redis for Python Developers** - Since you have Python experience and have completed RU101, this intermediate course will further your skills in building applications with Redis. This course typically takes 6-8 hours

with_retrieval strategy:
Becoming proficient in Redis can vary depending on your learning pace and dedication, but with your background and interests, here's a potential pathway based on the courses available:

1. **RU101: Introduction to Redis Data Structures (Completed)** - You've already completed this foundational course, which covers the basic data structures in Redis.

2. **RU201: Redis for Python Developers** - This intermediate course will take about 6-8 hours. Since you have 2 years of Python experience and have completed RU101, you're well-prepared to dive into this course. This will enhance your skills in building Redis applications specifically using Python.

3. **RU301: Vector Similarity Search with Redis** - This advanced course (8-10 hours) requires completion
```

### Analyzing Context Strategy Results

Let's analyze what happened when we tested the same queries with different amounts of context.

#### What We Observed

**Query 1: "What course should I take next?"**

- **Minimal (system only):** Asked clarifying questions - "What's your background? What are your goals?"
- **With user context:** Immediately recommended RU201 based on Sarah's Python background and completed RU101
- **With retrieval:** Same recommendation PLUS specific course details (duration, topics) for better decision-making

**Query 2: "I want to learn about vector search"**

- **Minimal:** Suggested RU301 but couldn't verify if prerequisites were met
- **With user context:** Built a learning path (RU201 → RU301) based on what Sarah already completed
- **With retrieval:** Same path with detailed justification for each step

**Query 3: "How long will it take to become Redis-proficient?"**

- **Minimal:** Listed all courses but repeated RU101 (which Sarah already finished)
- **With user context:** Calculated time starting from RU201, acknowledging completed work
- **With retrieval:** Most accurate timeline with specific hours per course

---

### Key Insights

**1. System Context Alone = Generic Bot**
- Must ask follow-up questions
- Can't personalize
- Wastes user time with back-and-forth

**2. Adding User Context = Personal Assistant**
- Knows who you are
- Skips unnecessary questions
- Tailors recommendations instantly

**3. Adding Retrieved Context = Expert Advisor**
- Provides specific details (hours, topics, prerequisites)
- Makes responses actionable
- Gives users everything needed to decide

---

### The Pattern
```
More Context = Less Back-and-Forth = Better Experience

Minimal:     User asks → AI asks clarifying questions → User answers → AI responds
                         (3-4 interactions to get an answer)

Rich:        User asks → AI responds with personalized, detailed answer
                         (1 interaction - done)
```

---

### When to Use Each Strategy

| Strategy | Best For | Example |
|----------|----------|---------|
| **Minimal** | New users, no history available | First-time visitor to your site |
| **With User** | Returning users, simple queries | "What should I do next?" |
| **With Retrieval** | Complex decisions, detailed planning | "Plan my learning path for the year" |

---

### What This Means for Production

**The Right Context Strategy Depends On:**

1. **Do you have user history?**
   - Yes → Include user context
   - No → Use minimal, ask questions

2. **Is the query complex?**
   - Yes → Retrieve specific details
   - No → User context might be enough

3. **Are you near token limits?**
   - Yes → Switch to minimal or summarize
   - No → Use rich context

**Simple Rule:** Start with rich context (all four types). Only reduce when you hit token limits or lack data.

---

### Action Items

Based on this test, you should:

1. **Always include user context** when available (massive quality improvement, low token cost)
2. **Retrieve context dynamically** based on what the query asks about (don't retrieve RU201 details for every question)
3. **Monitor token usage** - several responses were cut off at 150 tokens
4. **Test with your own use case** - Run this experiment with your domain and queries

**Bottom Line:** More relevant context = better responses. The challenge is determining what's "relevant" and managing token budgets.

---
## 📚 Course Summary: What You've Learned

Congratulations! You've completed Chapter 1: Foundations of Context Engineering. Let's recap your journey.

### From Notebook 01: Why Context Engineering Matters

You discovered the fundamental problem that context engineering solves:

**The Core Problem:**
- AI agents without context are like doctors without medical records - they can't remember, personalize, or maintain coherent interactions
- This leads to frustrated users, operational inefficiency, and limited AI capabilities

**The Impact:**
- You saw the dramatic difference between context-less and context-aware AI through the university advisor example
- Without context: repetitive, generic, frustrating interactions
- With context: personalized, coherent, valuable assistance

**The Four Context Types:**
You learned the foundational framework:
1. **System Context** - "What am I?" (Role, rules, domain knowledge)
2. **User Context** - "Who are you?" (Profile, preferences, history)
3. **Conversation Context** - "What have we discussed?" (Dialogue flow)
4. **Retrieved Context** - "What information is relevant?" (On-demand data)

**The Fundamental Constraint:**
- Every AI model has a context window limit (e.g., 128K tokens)
- Every token spent on one type of context is unavailable for another
- Context engineering is optimization within constraints

**Real-World Importance:**
- Customer support, healthcare, sales, research - all require proper context management
- Poor context management has measurable business impact: 40-60% abandonment rates, 3-5x more interactions needed, high escalation rates

### From Notebook 02: How to Implement Context Engineering

You mastered the practical implementation:

**Hands-On Skills Acquired:**

1. **Building System Context**
   - How to define AI role and identity
   - Structuring domain knowledge effectively
   - Writing clear behavioral guidelines
   - Understanding static vs. dynamic information

2. **Creating User Context**
   - Storing user profiles as structured data
   - Formatting user information for LLMs
   - Personalizing responses based on user attributes
   - Seeing how different users get different context

3. **Managing Conversation Context**
   - Maintaining dialogue history across turns
   - Enabling natural reference resolution ("that course")
   - Building coherent multi-turn conversations
   - Strategies for handling long conversations

4. **Retrieving Dynamic Context**
   - Fetching relevant information on-demand
   - Query-specific data retrieval
   - Optimizing for relevance vs. completeness
   - Simulating database and search operations

**Integration Mastery:**
- You learned how to combine all four context types into a single LLM call
- You saw the complete message array structure that makes intelligent responses possible
- You understood how each context type contributes to the final response quality

**Strategic Thinking:**
You explored three context management strategies:
- **Minimal Context** - For new users with no history
- **Rich Context** - For returning users with established profiles
- **Optimized Context** - For long conversations near token limits

**Best Practices:**
1. Start simple, add complexity gradually
2. Measure token usage continuously
3. Optimize for relevance, not completeness
4. Use clear, structured formatting
5. Test and iterate based on results

### What You Can Do Now

After completing these two notebooks, you have the foundational skills to:

 - **Understand** why context engineering is critical for production AI systems  
 - **Identify** which context type to use for different information  
 - **Build** context-aware AI agents from scratch  
 - **Format** context appropriately for LLM consumption  
 - **Combine** multiple context sources into coherent requests  
 - **Optimize** token usage within context window constraints  
 - **Adapt** context strategies based on user type and conversation length  
 - **Implement** the Redis University course advisor pattern for your own domain  



## 🤔 What's Next?

In the next section, you'll dive deeper into advanced techniques:

**Section 2: Retrieved Context Engineering**
- **Notebook 1:** RAG Fundamentals and Implementation
  - Vector embeddings and semantic search with Redis
  - Building production-ready RAG pipelines
  - Why context quality matters
- **Notebook 2:** Engineering Context for Production
  - Data engineering workflows for context
  - Chunking strategies with LangChain
  - Production pipeline architectures
  - Quality optimization techniques
- Building production RAG systems with LangChain and LangGraph
- Semantic retrieval strategies
- Hybrid search approaches
- Optimizing retrieval performance

**Section 3: Memory Systems for Context Engineering**
- Long-term memory systems with Redis Agent Memory Server
- Working memory vs. long-term memory patterns
- Memory summarization and compression
- Multi-agent memory coordination

**Section 4: Production Optimization**
- Context compression techniques
- Caching strategies
- Performance monitoring
- Cost optimization

---

## 📚 Additional Resources

### **Context Engineering Fundamentals**
- [Prompt Engineering Guide](https://www.promptingguide.ai/) - Comprehensive guide to prompt engineering
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering) - Best practices
- [Anthropic's Guide to Building Effective Agents](https://www.anthropic.com/research/building-effective-agents) - Agent design patterns

### **LLM Context Management**
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction) - Framework for context-aware applications
- [Context Window Management](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) - Understanding token limits
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference) - Complete API documentation

### **Academic Papers and Technical Reports**
- [In-Context Learning Survey](https://arxiv.org/abs/2301.00234) - Research on how LLMs use context
- [Retrieval-Augmented Generation](https://arxiv.org/abs/2005.11401) - Foundational RAG paper
- [Lost in the Middle](https://arxiv.org/abs/2307.03172) - How LLMs use long contexts
- [Context Rot](https://github.com/chroma-core/context-rot?tab=readme-ov-file) -  How Increasing Input Tokens Impacts LLM Performance

### **Redis Resources**
- [Redis Documentation](https://redis.io/docs/) - Official Redis documentation
- [Redis University](https://university.redis.com/) - Free Redis courses
- [Redis Python Client](https://redis-py.readthedocs.io/) - redis-py documentation

---